<a href="https://colab.research.google.com/github/Tsmoore002900/deep-learning-challenge/blob/main/AlphabetSoupCharity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
url = "https://raw.githubusercontent.com/Tsmoore002900/deep-learning-challenge/main/Resources/charity_data.csv"
application_df = pd.read_csv(url)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns = ['EIN', 'NAME'], inplace=True)
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
for i in application_df.columns:
    print(i, len(application_df[i].unique()))

APPLICATION_TYPE 17
AFFILIATION 6
CLASSIFICATION 71
USE_CASE 5
ORGANIZATION 4
STATUS 2
INCOME_AMT 9
SPECIAL_CONSIDERATIONS 2
ASK_AMT 8747
IS_SUCCESSFUL 2


In [4]:
# Look at APPLICATION_TYPE value counts for binning
APPLICATION_TYPE = application_df['APPLICATION_TYPE'].value_counts()
APPLICATION_TYPE

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(APPLICATION_TYPE[APPLICATION_TYPE<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
app_df = pd.get_dummies(application_df)
app_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = app_df.drop('IS_SUCCESSFUL', axis=1).values
y = app_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(25724, 43)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.5706 - accuracy: 0.7182
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5558 - accuracy: 0.7295
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5525 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5508 - accuracy: 0.7303
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7296
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7323
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7323
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5460 - accuracy: 0.7338
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5459 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5612 - accuracy: 0.7286 - 316ms/epoch - 1ms/step
Loss: 0.5611904263496399, Accuracy: 0.7286297082901001


In [18]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

# New Section

## 3 Attempts to Optimize the Model

## Attempt 1

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_2 = tf.keras.models.Sequential()
# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))
# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                3520      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 30)                930       
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,911
Trainable params: 6,911
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_2 = nn_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5701 - accuracy: 0.7187
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5548 - accuracy: 0.7291
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5515 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7308
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7323
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7320
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7334
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5451 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5775 - accuracy: 0.7291 - 572ms/epoch - 2ms/step
Loss: 0.5775392055511475, Accuracy: 0.7290962338447571


## Attempt 2

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_3 = tf.keras.models.Sequential()
# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=X_train_scaled.shape[1]))
# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Forth hidden layer
nn_3.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Fifth hidden layer
nn_3.add(tf.keras.layers.Dense(units=30, activation='relu'))
# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 80)                3520      
                                                                 
 dense_8 (Dense)             (None, 30)                2430      
                                                                 
 dense_9 (Dense)             (None, 30)                930       
                                                                 
 dense_10 (Dense)            (None, 30)                930       
                                                                 
 dense_11 (Dense)            (None, 30)                930       
                                                                 
 dense_12 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,771
Trainable params: 8,771
Non-traina

In [ ]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_3 = nn_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7403
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5324 - accuracy: 0.7414
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7413
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7404
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5329 - accuracy: 0.7403
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5326 - accuracy: 0.7409
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7411
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5324 - accuracy: 0.7402
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5323 - accuracy: 0.7411
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5327 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.7010 - accuracy: 0.4594 - 827ms/epoch - 3ms/step
Loss: 0.7010330557823181, Accuracy: 0.4593586027622223


## Attempt 3

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_4 = tf.keras.models.Sequential()
# First hidden layer
nn_4.add(tf.keras.layers.Dense(units=80, activation='tanh', input_dim=X_train_scaled.shape[1]))
# Second hidden layer
nn_4.add(tf.keras.layers.Dense(units=30, activation='tanh'))
# Third hidden layer
nn_4.add(tf.keras.layers.Dense(units=30, activation='tanh'))
# Output layer
nn_4.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn_4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                3520      
                                                                 
 dense_14 (Dense)            (None, 30)                2430      
                                                                 
 dense_15 (Dense)            (None, 30)                930       
                                                                 
 dense_16 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,911
Trainable params: 6,911
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn_4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_4 = nn_4.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5692 - accuracy: 0.7201
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5567 - accuracy: 0.7294
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7306
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7305
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7316
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7328
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7314
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5465 - accuracy: 0.7329
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7315
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5442 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5960 - accuracy: 0.7293 - 362ms/epoch - 1ms/step
Loss: 0.5960044860839844, Accuracy: 0.7293294668197632
